In [ ]:
# ===== Pacotes / Packages =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
# ===== 1. Ler os dados .XLSX / Read .XLSX data =====
dados = pd.read_excel("NAME.xlsx")
print(dados.head())

In [ ]:
# === Ler os dados .CSV / Read .CSV data ===
dados = pd.read_csv('NAME.csv', sep=',')  # especifica o separador / specifies the separator
print(dados.head())

In [ ]:
# ===== 2. Garantir que preditoras sejam numéricas / Ensure predictors are numeric =====
X = dados.drop(dados.columns[0], axis=1).apply(pd.to_numeric)
y = dados[dados.columns[0]].values.ravel()  # Classe na primeira coluna / Class in first column

# ===== 3. Padronizar os dados / Standardize data =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===== Exclui NaN / Excludes NaN =====
dados_cleaned = dados.dropna()

In [ ]:
# ===== 4. Divisão treino/teste / Training/testing division =====
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=1234, stratify=y
)

print("Train:", X_train.shape, y_train.shape)
print("Test:", X_test.shape, y_test.shape)

In [ ]:
# ===== 5. Treinar SVM / Train SVM =====
modelo_svm = SVC(kernel="linear", probability=True, random_state=1234) #testar com outros kernel / test with other kernels = linear, poly, rbf e sigmoid.
modelo_svm.fit(X_train, y_train)

In [ ]:
# ===== 6. Predição / Prediction =====
y_pred_train = modelo_svm.predict(X_train)
y_pred_test = modelo_svm.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Test Accuracy:", accuracy_score(y_test, y_pred_test))

In [ ]:
# ===== 7. Métricas / Metrics =====
classes = sorted(np.unique(y))

# Matriz de confusão / Confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_test, labels=classes)
cm_df = pd.DataFrame(conf_mat, index=classes, columns=classes)

# Visualização / Preview
plt.figure(figsize=(6,5))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predict")
plt.ylabel("True")
plt.title("Confusion Matrix - SVM")
plt.show()

# Classification report
print("\nClassification Report Teste:\n")
print(classification_report(y_test, y_pred_test, target_names=[str(c) for c in classes]))

In [ ]:
# ===== 8. Validação Cruzada com K-Fold / K-Fold Cross-Validation =====


# Validar o modelo SVM com validação cruzada (K=5 por padrão) / Validate the SVM model with cross-validation (K=5 by default)
cv_scores = cross_val_score(modelo_svm, X_scaled, y, cv=5)

print("Accuracies per Fold:", cv_scores)
print("Average Accuracy (K-Fold):", np.mean(cv_scores))
print("Standard Deviation of Accuracy (K-Fold):", np.std(cv_scores))

In [ ]:
# ===== 9. Feature Importance (para kernel linear) =====

# Extrair os coeficientes do modelo (para kernel linear) / Extract model coefficients (for linear kernel)
# Os coeficientes indicam a importância de cada feature na decisão de fronteira / The coefficients indicate the importance of each feature in the frontier decision
coeficientes = modelo_svm.coef_[0]  # Pegando os coeficientes para a primeira classe (ajustar se necessário) / Getting the coefficients for the first class (adjust if necessary)

# Obter os nomes das features / Get feature names
nomes_features = X.columns

# Criar um DataFrame para visualizar coeficientes e nomes das features / Create a DataFrame to visualize coefficients and feature names
feature_importance = pd.DataFrame({
    'feature': nomes_features,
    'importance': np.abs(coeficientes) # Usar valor absoluto para importância / Use absolute value for importance
})

# Ordenar as features por importância / Sort features by importance
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Visualizar as top N features (ex: top 10) / View the top N features (e.g. top 10)
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20), palette='viridis')
plt.title('Top 10 Features by Importance (Linear SVM Coefficients)')
plt.xlabel('Importance (Absolute Coefficient Value)')
plt.ylabel('Feature')
plt.show()

In [ ]:
# ===== 10. Plot classification boundaries =====

def plot_decision_boundary_multiclass(model, X, y, title="SVM Decision Boundary (PCA 2D)"):
    # Reduz para 2D com PCA / Reduce to 2D with PCA
    pca = PCA(n_components=2)
    X_2d = pca.fit_transform(X)

    # Converter rótulos de classe para numéricos para o scatter plot / Convert class labels to numeric for scatter plot
    y_numeric, classes = pd.factorize(y)

    # Modelo auxiliar só para visualização / Auxiliary model for viewing only
    viz_model = SVC(kernel=model.kernel, C=model.C, gamma=model.gamma, decision_function_shape='ovr')
    viz_model.fit(X_2d, y_numeric) # Treinar o modelo auxiliar com rótulos numéricos / Train auxiliary model with numeric labels


    fig, ax = plt.subplots(figsize=(6, 5))

    # Fronteira de decisão (somente 'predict') / Decision boundary ('predict' only)
    DecisionBoundaryDisplay.from_estimator(
        viz_model,
        X_2d,
        response_method="predict",
        plot_method="pcolormesh",
        cmap="coolwarm",
        alpha=0.3,
        ax=ax
    )

    # Pontos / Points
    scatter = ax.scatter(X_2d[:, 0], X_2d[:, 1], c=y_numeric, s=30, edgecolors="k", cmap="coolwarm") # Usar y_numeric para colorir os pontos / Use y_numeric to color the points

    # Vetores de suporte / Support vectors
    if hasattr(viz_model, "support_vectors_"):
        ax.scatter(
            viz_model.support_vectors_[:, 0],
            viz_model.support_vectors_[:, 1],
            s=150,
            facecolors="none",
            edgecolors="k"
        )

    # Criar um mapeamento manual para a legenda com os rótulos originais / Create a manual mapping for the legend with original labels
    handles, _ = scatter.legend_elements()
    legend = ax.legend(handles, classes, loc="upper right", title="Classes")
    ax.add_artist(legend)


    ax.set_title(title)
    plt.show()

# Usar no seu modelo já treinado (com 9 features), mas reduzindo p/ PCA 2D só no gráfico / Use on your already trained model (with 9 features), but reducing to 2D PCA only on the graph
plot_decision_boundary_multiclass(modelo_svm, X_scaled, y, "SVM Decision Boundary (Multiclass, PCA 2D)")